Working jupyter to create function to create a list of aminoacids likely to be used for mutation

In [20]:
path_pdb = './data/pdbs'
path_pqr=  './data/pqrs'
pdb_file = 'AF-C0H3V2-F1.pdb'
pqr_file = 'P39846.pqr'
output_path = './data/test'
name = pqr_file.split('.')[0]
from function import functional_aa
from function import free_aa
from helper_function import pdb2AA

functional_aas = functional_aa(path_pdb, pdb_file, output_path, df =False)

free_aas = free_aa(path_pdb, pdb_file, functional_aas)
AA = pdb2AA(path_pdb, pdb_file)

Pqr file already exists
Salt_bridge finished
H_bond finished
VdW_interaction finished


c:\Users\marik\OneDrive - bwedu\Uni HD\FS 4\Bioinfo\topic04_02\helper_function.py:71: RuntimeWarning: invalid value encountered in cast
  distance[:,0] = distance[:,0].astype('int')
c:\Users\marik\OneDrive - bwedu\Uni HD\FS 4\Bioinfo\topic04_02\helper_function.py:89: RuntimeWarning: invalid value encountered in arccos
  theta = np.arccos((d_DH[n,1,1]**2 + d_HA[n,1:]**2 - d_DA[1:,1:][n,:]**2)/(2*d_DH[n,1,1]*d_HA[1:,1:][n,:]))
c:\Users\marik\OneDrive - bwedu\Uni HD\FS 4\Bioinfo\topic04_02\helper_function.py:106: RuntimeWarning: invalid value encountered in cast
  array[:,0] = array[:,0].astype('int')


In [6]:
print(AAA2A(list(free_aas[:,1])))

MQVLAKENIKLNQTVSSKEEAIKLAGQTLIDNGYVTEDYISKMFEREETSSTFMGNFIAIPHGTEEAKSEVLHSGISIIQIPEGVEYGEGNTAKVVFGIAGKNNEHLDILSNIAIICSEEENIERLISAKSEEDLIAIFNEVN


In [5]:
print(list(free_aas[:,1]))

['MET', 'GLN', 'VAL', 'LEU', 'ALA', 'LYS', 'GLU', 'ASN', 'ILE', 'LYS', 'LEU', 'ASN', 'GLN', 'THR', 'VAL', 'SER', 'SER', 'LYS', 'GLU', 'GLU', 'ALA', 'ILE', 'LYS', 'LEU', 'ALA', 'GLY', 'GLN', 'THR', 'LEU', 'ILE', 'ASP', 'ASN', 'GLY', 'TYR', 'VAL', 'THR', 'GLU', 'ASP', 'TYR', 'ILE', 'SER', 'LYS', 'MET', 'PHE', 'GLU', 'ARG', 'GLU', 'GLU', 'THR', 'SER', 'SER', 'THR', 'PHE', 'MET', 'GLY', 'ASN', 'PHE', 'ILE', 'ALA', 'ILE', 'PRO', 'HIS', 'GLY', 'THR', 'GLU', 'GLU', 'ALA', 'LYS', 'SER', 'GLU', 'VAL', 'LEU', 'HIS', 'SER', 'GLY', 'ILE', 'SER', 'ILE', 'ILE', 'GLN', 'ILE', 'PRO', 'GLU', 'GLY', 'VAL', 'GLU', 'TYR', 'GLY', 'GLU', 'GLY', 'ASN', 'THR', 'ALA', 'LYS', 'VAL', 'VAL', 'PHE', 'GLY', 'ILE', 'ALA', 'GLY', 'LYS', 'ASN', 'ASN', 'GLU', 'HIS', 'LEU', 'ASP', 'ILE', 'LEU', 'SER', 'ASN', 'ILE', 'ALA', 'ILE', 'ILE', 'CYS', 'SER', 'GLU', 'GLU', 'GLU', 'ASN', 'ILE', 'GLU', 'ARG', 'LEU', 'ILE', 'SER', 'ALA', 'LYS', 'SER', 'GLU', 'GLU', 'ASP', 'LEU', 'ILE', 'ALA', 'ILE', 'PHE', 'ASN', 'GLU', 'VAL', 'ASN'

In [10]:
def fasparse(faspath):
    import numpy as np
    helixl = []
    sheetl = []
    data = np.loadtxt(faspath, dtype={'names': ('index', 'col1', 'col2', 'val1', 'val2', 'val3'),'formats': ('i4', 'S1', 'S1', 'f4', 'f4', 'f4')})
    indiceshelix = data['index'][data['col2'] == b'H']
    indicessheet = data['index'][data['col2'] == b'E']
    helix = np.split(indiceshelix, np.where(np.diff(indiceshelix) != 1)[0]+1)
    sheet = np.split(indicessheet, np.where(np.diff(indicessheet) != 1)[0]+1)
    return [helix, sheet]

In [15]:
import os
def AA2s4pred (directory_S4pred, output_path, AA_seq, prot):
    import os
    os.getcwd()
    # call s4pred and create fas file
    fastapath = os.path.join(output_path,f'{prot}.fasta')
    faspath = os.path.join(output_path,f'{prot}.fas')
    
    if os.path.isfile(fastapath):
        print(f'fasta file already exists {fastapath}')
    else:
        with open(os.path.join(output_path,f'{prot}.fasta'), "w") as fasta_file:
            fasta_file.write(f">{prot}\n{AA_seq}\n")
        print(f'fasta file created {fastapath}')
    if os.path.isfile(faspath):
        print('fas file already exists')
    else:
        os.chdir(directory_S4pred)
        os.system(f'python3 run_model.py "{fastapath}" > "{faspath}"')
        os.chdir('../../')
     
    #read fas file and   
    sec_pred = fasparse(faspath)    

    return sec_pred

In [8]:
def prot_mut(pdb_path, pdb_file, pqr_output_path, deep_mut=None, iterations=int, seed=None):
    
    import os
    from helper_function import pdb2AA
    from function import functional_aa
    from function import free_aa
    
    
    # top 20 features that correlate with melt point (positive and negative), based on prokaryotes323 columns, cutoff at +/- 0.32
    ideal_pos_value = 0.2
    ideal_neg_value = 0
    pos_corr = {'YR': 0.492442, 
                'RP': 0.480594, 
                'RG': 0.442533, 
                'R': 0.432905, 
                'WR': 0.392523, 
                'YP': 0.390709, 
                'LR': 0.386596, 
                'FR': 0.376553, 
                'VR': 0.370241, 
                'ER': 0.369174, 
                'RC': 0.357052, 
                #'Rhelix': 0.349204, 
                'MR': 0.343496, 
                'P': 0.340082, 
                'PG': 0.338434, 
                #'EAmotif': 0.332202, 
                'LP': 0.330337, 
                'EP': 0.328569, 
                'RH': 0.326193, 
                'AR': 0.324355, 
                #'ARmotif': 0.3241, 
                'NR': 0.323378}
    
    neg_corr = {'QT': -0.52935, 
                'MQ': -0.507329, 
                'QS': -0.502697, 
                'QC': -0.493738, 
                'Q': -0.469765, 
                'QD': -0.466556, 
                'QH': -0.455041, 
                'NQ': -0.435562, 
                'IQ': -0.429683, 
                'FQ': -0.42363, 
                'WQ': -0.420057, 
                'QK': -0.41872, 
                #'PolarAA': -0.406035, 
                'ST': -0.396185, 
                #'Qhelix': -0.37895, 
                'TC': -0.364763, 
                'MT': -0.359458, 
                'TH': -0.346921, 
                'TD': -0.34469, 
                'SH': -0.327684, 
                'SC': -0.321686, 
                'T': -0.321208}
    sorted_freq_pos = ['R', 'P', 'L', 'Y', 'E', 'G', 'A', 'V', 'M', 'F', 'W', 'N', 'H', 'C']
    sorted_freq_neg = ['K', 'N', 'W', 'F', 'I', 'D', 'M', 'C', 'H', 'S', 'T', 'Q']
    pos_corr_list = list(pos_corr.keys())
    neg_corr_list = list(neg_corr.keys())

    #https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003674
    conserv_subst = {
        'A': ['D', 'E', 'G', 'S', 'T'],
        'C': ['G', 'R', 'S', 'W', 'Y'],
        'D': ['A', 'E', 'G', 'H', 'N', 'V', 'Y'],
        'E': ['A', 'D', 'G', 'K', 'Q', 'V'],
        'F': ['I', 'L', 'Y'],
        'G': ['A', 'C', 'D', 'E', 'R'],
        'H': ['D', 'L', 'N', 'P', 'Q', 'R', 'Y'],
        'I': ['F', 'L', 'M', 'N', 'V'],
        'K': ['E', 'M', 'N', 'Q', 'R', 'T'],
        'L': ['F', 'H', 'I', 'M', 'P', 'Q', 'R', 'V', 'W'],
        'M': ['I', 'K', 'L', 'R', 'T', 'V'],
        'N': ['D', 'H', 'I', 'K', 'S', 'T', 'Y'],
        'P': ['H', 'L', 'Q', 'R', 'S'],
        'Q': ['E', 'H', 'K', 'L', 'P', 'R'],
        'R': ['C', 'G', 'H', 'K', 'L', 'M', 'P', 'Q', 'T', 'W'],
        'S': ['A', 'C', 'N', 'P', 'T', 'W', 'Y'],
        'T': ['A', 'K', 'M', 'N', 'R', 'S'],
        'V': ['D', 'E', 'I', 'L', 'M'],
        'W': ['C', 'L', 'R', 'S'],
        'Y': ['C', 'D', 'F', 'H', 'N'],
        }

    non_conservative_substitutions = {
        'A': ['P', 'V'],
        'C': ['F'],
        'F': ['C', 'S', 'V'],
        'G': ['S', 'V', 'W'],
        'I': ['K', 'R', 'S', 'T'],
        'K': ['I'],
        'L': ['S'],
        'P': ['A', 'T'],
        'Q': ['E', 'K'],
        'R': ['I', 'S'],
        'S': ['F', 'G', 'I', 'L', 'R'],
        'T': ['I', 'P'],
        'V': ['A', 'F', 'G'],
        'W': ['G'],
        }

    #extract aminoacid list
    aa_list = pdb2AA(pdb_path, pdb_file)
    aa_locked = functional_aa(pdb_path, pdb_file, pqr_output_path)
    aa_free = free_aa(pdb_path, pdb_file, aa_locked)
    
    # calculate ratio of WT protein for pos_corr and neg_corr and sort them in descending order
    from function import rel_aa_comp
    from function_mut import diff_weighted
    WT_dev_sum, WT_dev = diff_weighted(pos_corr, neg_corr, aa_list, ideal_pos_value, ideal_neg_value)
     
     
    #mutation
    #first_entry = list(WT_dev.items())[0]
     
            
    return WT_dev_sum, WT_dev

In [21]:
sum, dev = prot_mut(path_pdb, pdb_file, path_pqr, deep_mut=None, iterations=100, seed=None)

Pqr file already exists
Salt_bridge finished
H_bond finished
VdW_interaction finished


c:\Users\marik\OneDrive - bwedu\Uni HD\FS 4\Bioinfo\topic04_02\helper_function.py:71: RuntimeWarning: invalid value encountered in cast
  distance[:,0] = distance[:,0].astype('int')
c:\Users\marik\OneDrive - bwedu\Uni HD\FS 4\Bioinfo\topic04_02\helper_function.py:89: RuntimeWarning: invalid value encountered in arccos
  theta = np.arccos((d_DH[n,1,1]**2 + d_HA[n,1:]**2 - d_DA[1:,1:][n,:]**2)/(2*d_DH[n,1,1]*d_HA[1:,1:][n,:]))
c:\Users\marik\OneDrive - bwedu\Uni HD\FS 4\Bioinfo\topic04_02\helper_function.py:106: RuntimeWarning: invalid value encountered in cast
  array[:,0] = array[:,0].astype('int')


In [29]:
print(dev)
test = {'R':4, 'RV':5}
if 'R' in test.keys():
    print(test['R'])

[('IQ', 0.16083916083916083), ('ST', 0.13286713286713286), ('QS', 0.11188811188811189), ('NQ', 0.1048951048951049), ('SH', 0.1048951048951049), ('EP', 0.09512536083916083), ('QK', 0.09090909090909091), ('SC', 0.09090909090909091), ('ER', 0.08700436083916083), ('QT', 0.07692307692307693), ('TD', 0.07692307692307693), ('R', 0.07259498601398602), ('MT', 0.06993006993006994), ('TH', 0.06993006993006994), ('RP', 0.06814677202797204), ('WR', 0.06451858601398602), ('YR', 0.06352336503496503), ('FQ', 0.06293706293706294), ('QD', 0.055944055944055944), ('TC', 0.055944055944055944), ('P', 0.05403038601398602), ('RC', 0.05043137902097902), ('MQ', 0.04895104895104895), ('QH', 0.04895104895104895), ('T', 0.04895104895104895), ('YP', 0.04317676503496503), ('QC', 0.03496503496503497), ('MR', 0.033734165034965034), ('RH', 0.03027356503496504), ('Q', 0.027972027972027972), ('WQ', 0.027972027972027972), ('FR', 0.026359551048951053), ('NR', 0.026233490909090912), ('AR', 0.01904508391608392), ('PG', 0.016

In [24]:
def mutator(free_AA, deviation,pos_corr_list, sorted_freq_pos, neg_corr_list, sorted_freq_neg, conserv_subst):
    first_entry = deviation[0][0]
    mut_list = []
    
    # determine possible substitutions if the first entry is a single amino acid
    if len(first_entry) == 1:
        
        if first_entry in pos_corr_list: #checks if amount of aminoacid should be increaed
            for i, aminoacid in enumerate(free_AA):
                    # Find valid substitutions for the current amino acid
                    AA_subst = conserv_subst[aminoacid]
                    # Check if the current amino acid is in the positive correlation list
                    if 'R' in AA_subst:
                        mut_list.append(aminoacid + i + 'R')
                        free_AA[i] = 'R'
                    elif 'P' in AA_subst:
                        mut_list.append(aminoacid + i + 'P')
                        free_AA[i] = 'P'
        
        elif first_entry in neg_corr_list: #checks if amount of aminoacid should be decreased
            for i, aminoacid in enumerate(free_AA):
                if aminoacid == first_entry:
                    AA_subst = conserv_subst[aminoacid]
                    #tries to substitute the aminoacid to the aminoacids that correlate most
                    if 'R' in AA_subst:
                        mut_list.append(aminoacid + i + 'R')
                        free_AA[i] = 'R'
                    elif 'P' in AA_subst:
                        mut_list.append(aminoacid + i + 'P')
                        free_AA[i] = 'P'
                    #mutates the aminoacid to one that does not negatively correlate (maybe should be taken out if too many mutations)
                    elif aminoacid in sorted_freq_pos:
                        for n in range(len(sorted_freq_pos)):
                            if sorted_freq_pos[n] in AA_subst:
                                mut_list.append(aminoacid + i + sorted_freq_pos[n])
                                free_AA[i] = sorted_freq_pos[n]
                                break
        else:
            print('No valid substitution found')
            return None


    # determine possible substitutions if the first entry is a pair of amino acids
    else:
        for i, aminoacid in enumerate(free_AA):
            AA_subst_01 = conserv_subst[first_entry[0]]
            AA_subst_02 = conserv_subst[first_entry[1]]
            
        
        
        
                
                
            
            
            
            

    
    
    for i, amino_acid in enumerate(free_AA):
        # Find valid substitutions for the current amino acid
        valid_subst = [sub for sub in conserv_subst[amino_acid] if sub in pos_corr_list[i]]
        if not valid_subst:
            valid_subst = [sub for sub in conserv_subst[amino_acid] if sub in neg_corr_list[::-1][i]]
            


SyntaxError: invalid syntax (2920631105.py, line 19)

In [ ]:
def substitute_amino_acids(amino_acid_string, substitutions, desired_types, preference_scores):
  """
  Performs amino acid substitutions aiming for desired types with preference for higher scores.

  Args:
      amino_acid_string (str): String containing the amino acids to be substituted (e.g., 'WY').
      substitutions (dict): Dictionary containing possible substitutions for each amino acid.
      desired_types (list): List of desired amino acid combinations (e.g., ['CL', 'CD', 'RF']).
      preference_scores (dict): Dictionary containing integer scores for each amino acid.

  Returns:
      str: The substituted amino acid string or None if no suitable substitution is found.
  """

  # Check if string length matches desired type length
  if len(amino_acid_string) != len(desired_types[0]):
      return None  # String length mismatch

  substituted_string = ""
  for i, amino_acid in enumerate(amino_acid_string):
      # Find valid substitutions for the current amino acid
      valid_substitutions = [sub for sub in substitutions[amino_acid] if sub in desired_types[i]]
      
      # Check if any valid substitutions exist
      if not valid_substitutions:
          return None  # No valid substitutions found

      # Choose the substitution with the highest preference score (break ties randomly)
      best_sub = random.choice([sub for sub in valid_substitutions if preference_scores[sub] == max(preference_scores[sub_choice] for sub_choice in valid_substitutions)])

      substituted_string += best_sub

  return substituted_string

In [9]:
from collections import Counter

pos_corr = {
    'AR': 0.324355, 
    'VR': 0.370241, 
    'LR': 0.386596, 
    'LP': 0.330337, 
    'MR': 0.343496, 
    'FR': 0.376553, 
    'WR': 0.392523, 
    'NR': 0.323378, 
    'YR': 0.492442, 
    'YP': 0.390709, 
    'ER': 0.369174,
    'EP': 0.328569,
    'RH': 0.326193,
    'RC': 0.357052,
    'RP': 0.480594,
    'RG': 0.442533,
    'PG': 0.338434,
    #'Rhelix': 0.349204,
    'R': 0.432905,
    'P': 0.340082,
    #'ARmotif': 0.3241,
    #'EAmotif': 0.332202
    }
neg_corr = {
    'IQ': -0.429683,
    'MQ': -0.507329,
    'MT': -0.359458,
    'FQ': -0.42363,
    'WQ': -0.420057,
    'NQ': -0.435562,
    'QS': -0.502697,
    'QT': -0.52935,
    'QD': -0.466556,
    'QH': -0.455041,
    'QK': -0.41872,
    'QC': -0.493738,
    'ST': -0.396185,
    'SH': -0.327684,
    'SC': -0.321686,
    'TD': -0.34469,
    'TH': -0.346921,
    'TC': -0.364763,
    #'Qhelix': -0.37895,
    'Q': -0.469765,
    'T': -0.321208,
    #'PolarAA': -0.406035
}
pos_all = ''.join(pos_corr.keys())
pos_counts = Counter(pos_all)

neg_all = ''.join(neg_corr.keys())
neg_counts = Counter(neg_all)
combined_dict = {key: pos_counts.get(key, 0) - neg_counts.get(key, 0) for key in set(pos_counts) | set(neg_counts)}
combined_dict = dict(sorted(combined_dict.items(), key=lambda item: item[1], reverse=True))
print(combined_dict)
print(pos_counts)
print(neg_counts)
print(list(pos_counts.keys()))
print(list(neg_counts.keys()))    

{'R': 14, 'P': 6, 'L': 2, 'Y': 2, 'E': 2, 'G': 2, 'V': 1, 'A': 1, 'W': 0, 'F': 0, 'N': 0, 'I': -1, 'M': -1, 'K': -1, 'C': -2, 'D': -2, 'H': -2, 'S': -4, 'T': -7, 'Q': -12}
Counter({'R': 14, 'P': 6, 'L': 2, 'Y': 2, 'E': 2, 'G': 2, 'A': 1, 'V': 1, 'M': 1, 'F': 1, 'W': 1, 'N': 1, 'H': 1, 'C': 1})
Counter({'Q': 12, 'T': 7, 'S': 4, 'H': 3, 'C': 3, 'M': 2, 'D': 2, 'I': 1, 'F': 1, 'W': 1, 'N': 1, 'K': 1})
['A', 'R', 'V', 'L', 'P', 'M', 'F', 'W', 'N', 'Y', 'E', 'H', 'C', 'G']
['I', 'Q', 'M', 'T', 'F', 'W', 'N', 'S', 'D', 'H', 'K', 'C']


Get list of pos/neg correlating features for Meltpoint

In [19]:
import pandas as pd
import os
pro_df: pd.DataFrame = pd.read_csv(os.path.join('./data', 'prokaryotes_323columns.csv'))
pro_df = pro_df.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'])

C:\Users\marik\AppData\Local\Temp\ipykernel_27712\768696474.py:3: DtypeWarning: Columns (8,21,22,23,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  pro_df: pd.DataFrame = pd.read_csv(os.path.join('./data', 'prokaryotes_323columns.csv'))


In [20]:
pro_corr = pro_df.corr(numeric_only=True)['meltPoint']
pos_corr = pro_corr[pro_corr > 0.32]
neg_corr = pro_corr[pro_corr < -0.32]

In [21]:
pos_corr = list(pro_corr.index)
neg_corr = list(neg_corr.index)
print(neg_corr)

['IQ', 'MQ', 'MT', 'FQ', 'WQ', 'NQ', 'QS', 'QT', 'QD', 'QH', 'QK', 'QC', 'ST', 'SH', 'SC', 'TD', 'TH', 'TC', 'Qhelix', 'Q', 'T', 'PolarAA']


Needs to be mutated

In [24]:
print(pro_corr[pro_corr < -0.32])

IQ        -0.429683
MQ        -0.507329
MT        -0.359458
FQ        -0.423630
WQ        -0.420057
NQ        -0.435562
QS        -0.502697
QT        -0.529350
QD        -0.466556
QH        -0.455041
QK        -0.418720
QC        -0.493738
ST        -0.396185
SH        -0.327684
SC        -0.321686
TD        -0.344690
TH        -0.346921
TC        -0.364763
Qhelix    -0.378950
Q         -0.469765
T         -0.321208
PolarAA   -0.406035
Name: meltPoint, dtype: float64
